In [ ]:
!pip install -q crewai langchain langchainhub langchain-openai openai gradio

import os
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import gradio as gr

# ⚠️ Replace with your actual OpenAI API Key
api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("🚨 OPENAI_API_KEY not found. Please add it via the key icon on the left.")

# ✅ Set it for OpenAI SDK
os.environ["OPENAI_API_KEY"] = api_key

# Use GPT-3.5 or GPT-4
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)

# 🔹 Define CrewAI Agents
resume_analyzer = Agent(
    role="Resume Analyzer",
    goal="Analyze the input resume for strengths and weaknesses",
    backstory="You're an expert in identifying valuable insights from resumes",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

skill_matcher = Agent(
    role="Skill Matcher",
    goal="Match resume skills with in-demand skills from current job market",
    backstory="You're up-to-date with industry trends and job requirements",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

career_recommender = Agent(
    role="Career Recommendation Expert",
    goal="Suggest best-suited career paths and recommend upskilling courses",
    backstory="You specialize in personalized career guidance based on user profiles",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 🔹 Function to run the CrewAI process
def analyze_resume(resume_text):
    # Define tasks
    task1 = Task(
        description=f"Analyze this resume and highlight strengths, weaknesses, and key areas: {resume_text}",
        expected_output="Detailed resume insights with strengths and gaps.",
        agent=resume_analyzer
    )

    task2 = Task(
        description=f"Match the identified skills with current job market trends and identify missing skills.",
        expected_output="List of matched and missing skills from job market.",
        agent=skill_matcher,
        context=[task1]
    )

    task3 = Task(
        description="Suggest 3 ideal job roles and 3 relevant online courses based on the resume analysis and market trends.",
        expected_output="Career suggestions and learning resources.",
        agent=career_recommender,
        context=[task1, task2]
    )

    # Create Crew
    crew = Crew(
        agents=[resume_analyzer, skill_matcher, career_recommender],
        tasks=[task1, task2, task3],
        verbose=True
    )

    # Run and return result
    result = crew.kickoff()
    return result

# 🔹 Gradio UI
demo = gr.Interface(
    fn=analyze_resume,
    inputs=gr.Textbox(lines=15, label="Paste Your Resume Text"),
    outputs=gr.Textbox(lines=25, label="Personalized Career Insights"),
    title="🎯 AI Career Coach with CrewAI",
    description="Upload your resume and get personalized job recommendations, skill gap analysis, and course suggestions.",
    theme="default"
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://802ad06b144cebd105.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
